# Carga de librerías y funciones

In [1]:
import json
import joblib

import numpy as np
import pandas as pd

from sklearn.ensemble import RandomForestRegressor

In [2]:
def get_filename(path):
    # Obtenemos el nombre del fichero
    filename = path.split("/")[-1].split(".")[0]
    return filename

# Carga de datos

In [3]:
# Ruta de los conjuntos de datos de entrenamiento y test
train_path = input("Conjunto de train: ")
test_path = input("Conjunto de test: ")

# Obtenemos el nombre del fichero de entrenamiento
filename = get_filename(train_path)

# Cargamos los conjuntos de datos
train = pd.read_csv(train_path, sep=",")
test = pd.read_csv(test_path, sep=",")

# Obtenemos las variables categóricas del conjunto de datos
cat_cols = ["ID_FINCA", "ID_ZONA", "ID_ESTACION", "VARIEDAD", "MODO", "TIPO", "COLOR"]
cat_cols_train = [col for col in cat_cols if col in train.columns]

# Cambiamos a categóricas las variables que son categóricas
train[cat_cols_train] = train[cat_cols_train].astype('category')
test[cat_cols_train] = test[cat_cols_train].astype('category')

# División de los datos en variables predictoras y variable objetivo
x_train = train[train.columns.drop(["PRODUCCION"])]
y_train = train.PRODUCCION
x_test = test[test.columns.drop(["PRODUCCION"])]

# Mostramos las primeras observaciones del conjunto de entrenamiento
x_train.head(5)

,ID_FINCA,ID_ZONA,ID_ESTACION,VARIEDAD,MODO,TIPO,COLOR,SUPERFICIE,TemperatureLocalDayAvg,TemperatureLocalNighttimeAvg,WindSpeedLocalDayAvg
0,48626,302,13,32,2,0,1,0.977041,0.245761,0.396521,-0.197433
1,47921,302,13,32,2,0,1,1.625634,0.245761,0.396521,-0.197433
2,87611,602,14,81,1,0,1,0.806398,-0.859527,-0.492986,0.020158
3,12257,215,14,17,1,0,1,0.353163,-0.859527,-0.492986,0.020158
4,5696,919,14,59,1,0,1,1.657182,-0.859527,-0.492986,0.020158


In [4]:
# Cargamos la media y la desviación estándar para deshacer la normalización de la predicción
mean_sd_df = pd.read_csv("mean_sd.txt")
mean, sd = mean_sd_df.PRODUCCION

# Entrenamiento del modelo

En nuestro caso hemos seleccionado como mejor modelo un RandomForestRegressor.

In [5]:
# Tipo de modelo
model = "random_forest"

# Cargamos los parámetros guardados del mejor modelo
with open(f"params_{model}_{filename}.json", "r") as f:
    params = json.load(f)

# Instanciamos el modelo Random Forest Regressor
regressor = RandomForestRegressor(**params, random_state=123)

# Entrenamos el modelo
regressor.fit(x_train, y_train)

# Guardamos el modelo en un fichero joblib
joblib.dump(regressor, f"{model}_{filename}.joblib")

['random_forest_train_train_norm_tempwind.joblib']

# Predicción

In [6]:
# Predecimos con el modelo
y_pred = regressor.predict(x_test)

# Deshacemos la normalización de la variable predicha
y_pred_t = y_pred * sd + mean

# Generación del dataset de entrega

In [7]:
# Cargamos el dataset que contiene las columnas necesarias para el dataset de entrega
df = pd.read_csv("UH_2023_TRAIN.txt", sep="|", decimal=".")
# Nos quedamos con la campaña 22, para la cual hemos predicho la producción
df = df[df.CAMPAÑA == 22].reset_index(inplace=False, drop=True)

# Nos quedamos únicamente con las columnas necesarias
df_final = df[["ID_FINCA", "VARIEDAD", "MODO", "TIPO", "COLOR", "SUPERFICIE"]]

# Añadimos la predicción como columna de producción
df_final.insert(6, "PRODUCCION", np.round(y_pred_t, 2))

# Ordenamos el dataset
df_final = df_final.sort_values(
    ["ID_FINCA", "VARIEDAD", "MODO", "TIPO", "COLOR", "SUPERFICIE"]
).reset_index(inplace=False, drop=True)

# Guardamos el dataset de entrega
df_final.to_csv("Unity_UH2023.txt", sep="|", decimal=".", index=False, header=False)

# Mostramos las primeras observaciones del dataset
df_final.head(5)

,ID_FINCA,VARIEDAD,MODO,TIPO,COLOR,SUPERFICIE,PRODUCCION
0,439,9,2,0,0,1.0800,3906.62
1,447,40,2,0,1,0.4694,2309.71
2,523,32,2,0,1,0.6478,1407.79
3,702,59,2,0,1,1.1331,4613.61
4,797,59,1,0,1,7.5143,19489.16
